In [5]:
!pip install scikit-optimize --user --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 6.1 MB/s eta 0:00:00


In [123]:
import pandas as pd
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, mean_squared_error, make_scorer, mean_absolute_error

import numpy as np

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import classification_report, r2_score

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding

from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

from sklearn.model_selection import GridSearchCV,train_test_split, cross_val_score, TimeSeriesSplit

from yellowbrick.classifier.rocauc import roc_auc
from yellowbrick.target.feature_correlation import feature_correlation
from yellowbrick.classifier import precision_recall_curve

import warnings

from skopt import BayesSearchCV

pd.set_option('display.precision', 3)

In [2]:
from IPython.display import display, HTML
show_html = lambda html: display(HTML(html))

In [6]:
df = pd.read_csv('international_matches_clean.csv')
df.head()

,away_team_continent,away_team_fifa_rank,away_team_goalkeeper_score,away_team_mean_defense_score,away_team_mean_midfield_score,away_team_mean_offense_score,away_team_total_fifa_points,home_team_continent,home_team_fifa_rank,home_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_midfield_score,home_team_mean_offense_score,home_team_total_fifa_points,shoot_out,result
0,0.4,0.349,0.729,0.652,0.707,0.601,0.0,0.4,0.012,0.938,0.842,0.898,0.907,0.0,0.0,0.0
1,0.4,0.032,0.812,0.961,0.977,0.871,0.0,0.4,0.517,0.708,0.536,0.473,0.496,0.0,0.0,0.0
2,0.4,0.397,0.521,0.399,0.374,0.480,0.0,0.4,0.140,0.583,0.664,0.610,0.640,0.0,0.0,3.0
3,0.4,0.212,0.604,0.360,0.612,0.601,0.0,0.4,0.459,0.604,0.315,0.341,0.597,0.0,0.0,-2.0
4,0.4,0.196,0.625,0.622,0.626,0.649,0.0,0.4,0.047,1.000,1.000,0.843,0.982,0.0,0.0,1.0


In [73]:
X = df[df.columns.difference(['result'])]
y = np.array(df['result'])

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# **Random Forest**

In [134]:
param = {'n_estimators': [5, 10, 25, 40, 50, 75, 100, 200], 
         'criterion': ['squared_error', 'absolute_error', 'friedman_mse'],
         'max_depth': [None, 1, 2, 3, 5, 8, 9,10,15],
         'min_samples_leaf': [1,2,3,5,10]}

cv = 3
iter = 40

rf = RandomForestRegressor(random_state=0);
rf_bs = BayesSearchCV(rf, param, n_iter=iter, cv=cv, n_jobs=-1, refit=True, random_state=0);
rf_bs.fit(X_train, y_train)

BayesSearchCV(cv=3, estimator=RandomForestRegressor(random_state=0), n_iter=40,
              n_jobs=-1, random_state=0,
              search_spaces={'criterion': ['squared_error', 'absolute_error',
                                           'friedman_mse'],
                             'max_depth': [None, 1, 2, 3, 5, 8, 9, 10, 15],
                             'min_samples_leaf': [1, 2, 3, 5, 10],
                             'n_estimators': [5, 10, 25, 40, 50, 75, 100, 200]})

In [135]:
show_html(pd.DataFrame(rf_bs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

,params,mean_test_score,rank_test_score
31,"{'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 2, 'n_estimators': 200}",0.170,1
37,"{'criterion': 'friedman_mse', 'max_depth': 5, 'min_samples_leaf': 10, 'n_estimators': 50}",0.169,2
18,"{'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 10, 'n_estimators': 40}",0.169,3
9,"{'criterion': 'friedman_mse', 'max_depth': 5, 'min_samples_leaf': 2, 'n_estimators': 100}",0.169,4
29,"{'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 50}",0.168,5


# **SVM con kernel RBF**

In [ ]:
param = {'C':10**np.linspace(-3,3,101), 'gamma':['scale','auto']}

cv = 3
niter = 40

rbsvc =  SVC(kernel='rbf', max_iter=25000, random_state=0)
rbsvc_gs = BayesSearchCV(rbsvc,param,n_iter=niter, cv=cv, n_jobs=-1, refit=True, random_state=0)
rbsvc_gs.fit(X_train, y_train.squeeze());

In [87]:
show_html(pd.DataFrame(rbsvc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

,params,mean_test_score,rank_test_score
19,"{'C': 1.9952623149688788, 'gamma': 'auto'}",0.285,1
2,"{'C': 1.513561248436207, 'gamma': 'auto'}",0.283,2
28,"{'C': 0.43651583224016566, 'gamma': 'scale'}",0.281,3
26,"{'C': 0.38018939632056126, 'gamma': 'scale'}",0.280,4
0,"{'C': 1.513561248436207, 'gamma': 'scale'}",0.279,5


# **MLP**

In [90]:
sdscaler = StandardScaler()

X_train_sd = sdscaler.fit_transform(X_train)
X_test_sd = sdscaler.transform(X_test)

In [128]:
param = {'hidden_layer_sizes':[10, 50, 100, 200], 
         'activation':['relu', 'logistic', 'identity'], 
         'learning_rate_init': [0.001, 0.01, 0.1]  }

mlp =  MLPRegressor(max_iter=10000, early_stopping=True, n_iter_no_change=20,learning_rate='adaptive',random_state=0)
mlp_gs =  GridSearchCV(mlp,param,cv=3, n_jobs=-1, refit=True)
mlp_gs.fit(X_train, y_train);

In [129]:
show_html(pd.DataFrame(mlp_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

,params,mean_test_score,rank_test_score
21,"{'activation': 'logistic', 'hidden_layer_sizes': 200, 'learning_rate_init': 0.001}",0.183,1
13,"{'activation': 'logistic', 'hidden_layer_sizes': 10, 'learning_rate_init': 0.01}",0.181,2
15,"{'activation': 'logistic', 'hidden_layer_sizes': 50, 'learning_rate_init': 0.001}",0.180,3
18,"{'activation': 'logistic', 'hidden_layer_sizes': 100, 'learning_rate_init': 0.001}",0.180,4
16,"{'activation': 'logistic', 'hidden_layer_sizes': 50, 'learning_rate_init': 0.01}",0.180,5
